In [21]:
for i in range(0, len([1,2]), 5):
    print([1,2][i : i + 5])

[1, 2]


In [13]:
from openai import OpenAI
from openai import APITimeoutError, InternalServerError, BadRequestError, RateLimitError, UnprocessableEntityError
from tenacity import (
    retry,
    retry_if_exception_type,
    stop_after_attempt,
    wait_exponential,
)

client = OpenAI()

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(min=2, max=5),
    retry=retry_if_exception_type(
        (
            APITimeoutError,
            InternalServerError,
            RateLimitError,
            UnprocessableEntityError
        )
    )
)
def say_hello():
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "user", "content": "HELLO"}
        ]
    )
    print(response)

In [15]:
for _ in range(5):
    say_hello()

ChatCompletion(id='chatcmpl-8iKsIOOKt9H0rrdoNfngIbWps4Q4k', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I help you today?', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1705577366, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_fe56e538d5', usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))
ChatCompletion(id='chatcmpl-8iKsIT894HjftXI1SCwzAtoh9ZlyX', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1705577366, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_fe56e538d5', usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))
ChatCompletion(id='chatcmpl-8iKsNUcEPQrFk5LXpSnNw8azIyERk', choices=[Choice(finish_reason='stop', index=0, message

In [8]:
response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=["Hello"]*20,
    max_tokens=20,
)
# for choice in response.choices:
#     print(choice)

In [10]:
response.usage

CompletionUsage(completion_tokens=388, prompt_tokens=20, total_tokens=408)

In [11]:
from openai import OpenAI
client = OpenAI()
# for _ in range(5):
response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt="Hello",
    max_tokens=20,
)

In [12]:
response.usage

CompletionUsage(completion_tokens=20, prompt_tokens=1, total_tokens=21)

In [2]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

from openai import AsyncOpenAI

async_client = AsyncOpenAI()

async def async_say_hello():
    response = await async_client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "user", "content": "HELLO"}
        ]
    )
    print(response)
    return response

In [ ]:
from tqdm.asyncio import tqdm

for f in tqdm.as_completed([async_say_hello() for _ in range(5)]):
    await f

In [ ]:
async def main():
    await asyncio.gather(*[async_say_hello() for _ in range(20)])

asyncio.run(main())

In [6]:
def _generate_batches(items: list, batch_size: int = 5):
    for i in range(0, len(items), batch_size):
        yield items[i : i + batch_size]

In [9]:
async def run():
    calls = [async_say_hello() for _ in range(15)]
    for batch in _generate_batches(calls):
        print("running batch")
        for future in asyncio.as_completed(batch):
            await future
            
asyncio.run(run())

In [16]:
async def run():
    calls = [async_say_hello() for _ in range(15)]
    for batch in _generate_batches(calls):
        print("running batch")
        for future in asyncio.as_completed(batch):
            yield await future
            # yield result  # yield the result instead of returning it

In [22]:
run()

<async_generator object run at 0x10aaacc80>

In [18]:
async for result in run():
    print("RESULT",result)  # process result

running batch
ChatCompletion(id='chatcmpl-8iPqcvFDJGnlSmsv18ACdFNsrvrzX', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I help you today?', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1705596482, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_fe56e538d5', usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))
ChatCompletion(id='chatcmpl-8iPqc2XLtDwpbrR94GC8Svu4FY1wq', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1705596482, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_fe56e538d5', usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))
RESULT ChatCompletion(id='chatcmpl-8iPqcvFDJGnlSmsv18ACdFNsrvrzX', choices=[Choice(finish_reason='st

In [23]:
from pydantic import BaseModel
import asyncio
import logging
from openai import AsyncOpenAI
from openai import APITimeoutError, InternalServerError, BadRequestError, RateLimitError, UnprocessableEntityError
from tenacity import (
    retry,
    retry_if_exception_type,
    stop_after_attempt,
    wait_exponential,
)

client = AsyncOpenAI()

    
class Llm(BaseModel):
    model: str = "gpt-3.5-turbo-instruct"
    batch_size: int = 10
    use_legacy: bool = True
    
    async def run_batch_completions(self, prompts, **kwargs):
        calls = [
            self._run_completions(prompt, **kwargs) 
            for prompt in prompts
        ]
        for batch in self._batches(calls):
            for future in asyncio.as_completed(batch):
                yield await future
    
    @retry(
        stop=stop_after_attempt(3),
        wait=wait_exponential(min=2, max=5),
        retry=retry_if_exception_type(
            (
                APITimeoutError,
                InternalServerError,
                RateLimitError,
                UnprocessableEntityError
            )
        )
    )
    async def _run_completions(self, prompt, **kwargs):
        if self.use_legacy:
            return await self._run_legacy_completions(prompt, **kwargs)
        else:
            return await self._run_chat_completions(prompt, **kwargs)
    
    async def _run_legacy_completions(self, prompt, **kwargs):
        logging.info("legacy completions uses 'gpt-3.5-turbo-instruct' model")
        return await client.completions.create(
            model="gpt-3.5-turbo-instruct",
            prompt=prompt,
            **kwargs,
        )
        
    async def _run_chat_completions(self, prompt, **kwargs):
        messages = {"user": "role", "content": prompt}
        return await client.chat.completions.create(
            model=self.model,
            messages=messages,
            **kwargs
        )
    
    def _batches(self, prompts: list):
        if self.use_legacy:
            # can send 20 prompts at once with legacy completions. 
            # See: https://platform.openai.com/docs/guides/rate-limits/batching-requests
            prebatched_prompts = [
                prompt_batch for prompt_batch in self._generate_batches(prompts, 20)
            ]
            yield from self._generate_batches(prebatched_prompts, self.batch_size)
        else:
            yield from self._generate_batches(prompts, self.batch_size)
    
    def _generate_batches(self, prompts, batch_size):
        for i in range(0, len(prompts), batch_size):
            yield prompts[i : i + batch_size]


In [26]:
llm = Llm()
prompts = ["hello world"] * 10
calls = [
    llm._run_completions(prompt) 
    for prompt in prompts
]
for batch in llm._generate_batches(calls, llm.batch_size):
    print(batch)
    for future in asyncio.as_completed(batch):
        await future

[<coroutine object Llm._run_completions at 0x10b775210>, <coroutine object Llm._run_completions at 0x10b7746d0>, <coroutine object Llm._run_completions at 0x10b775300>, <coroutine object Llm._run_completions at 0x10b7745e0>, <coroutine object Llm._run_completions at 0x10b7754e0>, <coroutine object Llm._run_completions at 0x10b774040>, <coroutine object Llm._run_completions at 0x10b774130>, <coroutine object Llm._run_completions at 0x10b774220>, <coroutine object Llm._run_completions at 0x10b774a90>, <coroutine object Llm._run_completions at 0x10b774b80>]


In [17]:
async def main():
    async for result in run():
        print("RESULT",result)  # process result

asyncio.run(main())

running batch
ChatCompletion(id='chatcmpl-8iPiZvyPnY9eavml1Ritj3Wm5J5Gq', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1705595983, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_fe56e538d5', usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))
RESULT ChatCompletion(id='chatcmpl-8iPiZvyPnY9eavml1Ritj3Wm5J5Gq', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1705595983, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_fe56e538d5', usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))
ChatCompletion(id='chatcmpl-8iPiZt8GUuHx1K0rnirMFKWsWrkVf', choices=[Choice(finish_reason='

In [10]:
asyncio.run(run())
    # print(f"RESULTS: {f}")

running batch
ChatCompletion(id='chatcmpl-8iPcmYXrvlc7iB0ExMGQjcJIJuD63', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I help you today?', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1705595624, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_fe56e538d5', usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))
ChatCompletion(id='chatcmpl-8iPcmhSsPNiEWR1ZuFBk2gWgKKZFe', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1705595624, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_fe56e538d5', usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))
ChatCompletion(id='chatcmpl-8iPcm0qtYmJ7oHjUrQKNvqnMrcbKi', choices=[Choice(finish_reason='stop', in

In [ ]:
calls = [async_say_hello() for _ in range(15)]
for batch in _generate_batches(calls):
    print("running batch")
    await asyncio.gather(*batch)

In [51]:
async def main():
    await asyncio.gather(*[async_say_hello() for _ in range(15)])

asyncio.run(main())

In [64]:
functions = cb.get_functions(paths[0])

In [23]:
prompt = """
Explain in MAXIMUM 10 TOKENS what the following function does:
"""

In [24]:
prompt += functions[0].content

In [25]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
#   response_format={ "type": "json_object" },
  messages=[
    {"role": "user", "content": prompt}
  ]
)
print(response.choices[0].message.content)

Sets system prompt as message in model instantiation.


In [32]:
MODEL_INFO = {
    "gpt-4-1106-preview": {"context": 128192, "inprice": 0.01, "outprice": 0.03},
    "gpt-4": {"context": 8192, "inprice": 0.03, "outprice": 0.06},
    "gpt-4-0613": {"context": 8192, "inprice": 0.03, "outprice": 0.06},
    "gpt-4-32k": {"context": 32000, "inprice": 0.06, "outprice": 0.12},
    "gpt-4-32k-0613": {"context": 32000, "inprice": 0.06, "outprice": 0.12},
    "gpt-3.5-turbo-1106": {"context": 16385, "inprice": 0.0010, "outprice": 0.0020},
    "gpt-3.5-turbo-instruct": {"context": 4096, "inprice": 0.0015, "outprice": 0.0020},
    "gpt-3.5-turbo": {"context": 4096, "inprice": 0.0015, "outprice": 0.0020},
    "gpt-3.5-turbo-0613": {"context": 4096, "inprice": 0.0015, "outprice": 0.0020},
    "gpt-3.5-turbo-16k": {"context": 16385, "inprice": 0.0030, "outprice": 0.0040},
    "gpt-3.5-turbo-16k-0613": {"context": 16385, "inprice": 0.0030, "outprice": 0.0040},
}

In [48]:
def calc_cost(intokens, outtokens, model):
    return round((
        intokens * MODEL_INFO[model]["inprice"] 
        + outtokens * MODEL_INFO[model]["outprice"]
    ) / 1000, 8)

In [49]:
import tiktoken
def count_tokens(text: str, model: str):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [50]:
def estimate_cost(cb, model):
    cost = 0
    for path in cb.get_modules_paths():
        for function in cb.get_functions(path):
            cost += calc_cost(intokens=count_tokens(function.content, model), outtokens=10, model=model)
    return cost

In [51]:
estimate_cost(cb,model="gpt-3.5-turbo-1106")

0.008659

In [53]:
metadata = []

for path in cb.get_modules_paths():
    for function in cb.get_functions(path):
        prompt = """
        Explain in MAXIMUM 10 TOKENS what the following function does:
        """
        prompt += function.content

        response = client.chat.completions.create(
          model="gpt-3.5-turbo-1106",
          messages=[
            {"role": "user", "content": prompt}
          ]
        )
        metadata.append(
          {
            "path": path, 
            "function": function.name, 
            "code": function.content, 
            "response": response.choices[0].message.content}
          )

In [59]:
from codeas.utils import write_yaml

write_yaml("metadata.yaml", {"output":metadata})

In [98]:
functions = cb.get_functions(paths[0])

In [71]:
def read_file_lines(path):
    with open(path, "r") as f:
        return f.readlines()


In [118]:
lines = read_file_lines(paths[0])

In [135]:
def get_file_line(identifier, include_args=False):
    if include_args:
        NotImplementedError()
    else:
        return lines[identifier.start_point[0]][:identifier.end_point[1]]

def get_name(identifier):
    return identifier.text.decode()

In [136]:
def get_identifier(node):
    for child in node.children:
        if child.type == "identifier":
            return child

In [140]:
def get_function_line(node):
    identifier = get_identifier(node)
    return get_file_line(identifier)

In [168]:
lines = []

for path in cb.get_modules_paths():
    for function in cb.get_functions(path):
        lines.append(function.node.start_point[0])

In [197]:
import os
os.path.abspath(metadata[0]["path"])

'/Users/manuelrenner/repos/codeas/src/codeas/thread.py'

In [199]:
def get_absolute_path(path):
    return os.path.abspath(path)

In [202]:
repo_map = {}
for function, line in zip(metadata, lines):
    id_ = function["function"] + "_" + str(line)
    abs_path = get_absolute_path(function["path"])
    if abs_path not in repo_map:
        repo_map[abs_path] = {
            id_: {
                "name": function["function"],
                "line": line,
                "type": "function_definition",
                "short_desc": function["response"]
            }
        }
    else:
        repo_map[abs_path][id_] = {
            "name": function["function"],
            "line": line,
            "type": "function_definition",
            "short_desc": function["response"]
        }

In [ ]:
repo_map

In [203]:
import json

with open("repo_map.json", "w") as f:
    json.dump(repo_map, f)

In [ ]:
repo_map['../../codeas/src/codeas/thread.py']

In [189]:
functions[0].node

<Node type=function_definition, start_point=(38, 4), end_point=(41, 79)>

In [178]:
def get_description(path, node):
    id_ = get_name(get_identifier(node)) + "_" + str(node.start_point[0])
    return repo_map[path][id_]["short_desc"]

In [180]:
def get_file_structure(path, add_description=True):
    root_node =cb.parse_root_node(path)
    file_structure = ""
    for child in root_node.children:
        if child.type == "function_definition":
            file_structure += get_function_line(child) 
            # if add_description:
            #     file_structure += "():\n\t... # " + get_description(path, child) + "\n"
            # else:
            #     file_structure += "():\n\t...\n"
        elif child.type == "class_definition":
            file_structure += get_function_line(child) + "():\n"
            for grandchild in child.children[-1].children: #children[-1] is the class block
                if grandchild.type == "function_definition":
                    file_structure += get_function_line(grandchild)
                    # if add_description:
                    #     file_structure += "():\n\t... # " + get_description(path, grandchild) + "\n"
                    # else:
                    #     file_structure += "():\n\t...\n"
    return file_structure

get_file_structure("../../codeas/src/codeas/thread.py")

TypeError: 'int' object is not subscriptable

'class Thread():\n'

In [122]:
identifiers = []
for element in code_elements:
    for child in element.children:
        if child.type == "identifier":
            identifiers.append({"start_line": child.start_point[0], "end_pos": child.end_point[1]})

In [123]:
lines[identifiers[0]["start_line"]]

'class Thread(BaseModel):\n'

In [126]:
for identifier in identifiers:
    print(lines[identifier["start_line"]][:identifier["end_pos"]])
# identifiers

class Thread


In [101]:
def get_identifier(lines, node):
    for child_node in node.children:
        if child_node.type == "identifier":
            return child_node.start_point[0], child_node.end_point[1]

In [99]:
line, pos = get_identifier_line(functions[0].node)

In [100]:
lines[line][:pos]

'    def model_post_init'

In [69]:
print(cb.get_file_structure(paths[0]))

# ../../codeas/src/codeas/thread.py
class Thread(BaseModel):
    def model_post_init(self, __context: Any) -> None:
    def run(self):
    def _add_context_to_messages(self):
    def _remove_context_from_messages(self):
    def trim_messages(self):
    def _run_messages(self):
    def check_messages_fit_context_window(self):
    def check_codebase_context_fits(self):
    def remove_oldest_messages(self):
    def _run_completion(self):
    def _parse_delta_content(self, delta, response):
    def _print_delta_content(self, delta):
    def _parse_delta_tools(self, delta, response):
    def _print_delta_tools(self, response, live):
    def run_calls(self, tool_calls: List[dict]):
    def call(self, tool_call: dict):
    def _get_function_args(self, tool_call: dict):
    def _print_call(self, function, args):
    def _print_call_success(self):
    def _print_call_error(self, e):
    def count_tokens_from_messages(self):
    def count_tokens(self, text: str):
    def add_message(self, messag

In [67]:
function_def = ""
for node in functions[0].node.children:
        print(node.text.decode())

async
def
model_post_init
(self, __context: Any)
->
None
:
# executed on model instantiation
if self.system_prompt is not None:
            self.add_message({"role": "system", "content": self.system_prompt})


[<Node type="async", start_point=(38, 4), end_point=(38, 9)>,
 <Node type="def", start_point=(38, 10), end_point=(38, 13)>,
 <Node type=identifier, start_point=(38, 14), end_point=(38, 29)>,
 <Node type=parameters, start_point=(38, 29), end_point=(38, 51)>,
 <Node type="->", start_point=(38, 52), end_point=(38, 54)>,
 <Node type=type, start_point=(38, 55), end_point=(38, 59)>,
 <Node type=":", start_point=(38, 59), end_point=(38, 60)>,
 <Node type=comment, start_point=(39, 8), end_point=(39, 41)>,
 <Node type=block, start_point=(40, 8), end_point=(41, 79)>]